In [ ]:
# Jupyter Notebook: EDA for Hospital Readmission Project

# Import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load synthetic dataset
df = pd.read_csv("../data/raw/hospital_data.csv")

# Display first rows
print("✅ First 5 rows of dataset:")
display(df.head())

# Basic info
print("\n📊 Dataset info:")
print(df.info())

# Summary statistics
print("\n📈 Summary statistics:")
display(df.describe())

# Check missing values
print("\n❗ Missing values per column:")
print(df.isnull().sum())

# Distribution of target variable
plt.figure(figsize=(6,4))
sns.countplot(x='readmission_30d', data=df)
plt.title('Readmission within 30 days')
plt.xlabel('Readmission (0=No, 1=Yes)')
plt.ylabel('Count')
plt.show()

# Distribution of numeric features
numeric_features = ["age", "length_of_stay", "previous_admissions", "num_medications", "comorbidity_score", "distance_to_hospital_km"]
df[numeric_features].hist(figsize=(12,8), bins=15)
plt.suptitle("Numeric Feature Distributions")
plt.show()

# Correlation heatmap
plt.figure(figsize=(8,6))
sns.heatmap(df[numeric_features + ["readmission_30d"]].corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()

# Gender vs Readmission
plt.figure(figsize=(6,4))
sns.countplot(x='gender', hue='readmission_30d', data=df)
plt.title('Readmission by Gender')
plt.show()

# Insurance type vs Readmission
plt.figure(figsize=(8,4))
sns.countplot(x='insurance_type', hue='readmission_30d', data=df)
plt.title('Readmission by Insurance Type')
plt.show()

# Check class balance
readmission_rate = df['readmission_30d'].mean()
print(f"\nℹ️ Readmission rate: {readmission_rate:.2%}")

# Optional: Save processed version for modeling
df.to_csv("../data/processed/hospital_data_processed.csv", index=False)
print("✅ Processed CSV saved to data/processed/")
# ==============================
# Feature Engineering
# ==============================

print("\n🔧 Starting Feature Engineering...")

# Length of stay ratio: length_of_stay relative to previous admissions
df['length_of_stay_ratio'] = df['length_of_stay'] / (df['previous_admissions'] + 1)

# Medication burden: number of medications relative to age
df['medication_burden'] = df['num_medications'] / (df['age'] + 1)

# High comorbidity flag
df['high_comorbidity'] = (df['comorbidity_score'] >= 5).astype(int)

# Distance category: near/far hospital
df['distance_category'] = pd.cut(df['distance_to_hospital_km'],
                                 bins=[0, 5, 20, 50],
                                 labels=['Near', 'Medium', 'Far'])

# Encode categorical variables
df = pd.get_dummies(df, columns=['gender', 'insurance_type', 'distance_category'], drop_first=True)

print("✅ Feature engineering complete. New features added:")
print(df.columns[-10:])  # Show last 10 features added

# Save processed and feature-engineered dataset
df.to_csv("../data/processed/hospital_data_features.csv", index=False)
print("✅ Feature-engineered CSV saved to data/processed/hospital_data_features.csv")
